In [ ]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performerErrorTypeTest_V1 as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

In [ ]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [ ]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [ ]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [ ]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [ ]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [ ]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return tokens

In [ ]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [ ]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [ ]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce-binary-accuracy"
        Input_Path = "Trianing\InputTxt\Split-500-reduce-binary-accuracy"
        model_for_training_org_path = "Model-for-training-org\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\weight-1vs1-v1"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        #get all txt file in input path
        target_max_len = 0
        token_num = 0

        #start training
        import DataGeneratorTrainErrorTypeEdition as DGTrain
        import DataGeneratorValidationErrorTypeEdition as DGValidation
        import DataBuffer as db
        from random import randrange
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #load source_max_len
        source_max_len = loadDictionary(Trained_model_Path + "/" + source_max_len_name)
        #load training models and their len
        x_train = loadTestTrainData(model_for_training_path + "/" + "x_train_0.npy")
        x_validation = loadTestTrainData(model_for_training_path + "/" + "x_validation_0.npy")
        
        buffer_train_num = len(x_train)
        buffer_val_train_num = len(x_validation)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              errNum=36,
                              token_num=len(source_token_dict),
                              embed_dim=32, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        
        #for output
        #for x
        #data_number and block_size = data num
        input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_train_num,
            "data_type": int,
            "block_size": buffer_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = data num
        """
        output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_train_num, buffer_train_num],
            "data_type": [int, int],
            "block_size": [buffer_train_num, buffer_train_num] 
            }
        """
        output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_train_num],
            "data_type": [int],
            "block_size": [buffer_train_num] 
            }
        
        #===========================================
        #for output
        #for x
        #data_number and block_size = validation data num
        
        validation_input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_val_train_num,
            "data_type": int,
            "block_size": buffer_val_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = validation data num
        """
        validation_output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_val_train_num, buffer_val_train_num],
            "data_type": [int, int],
            "block_size": [buffer_val_train_num, buffer_val_train_num] 
            }
        """
        
        validation_output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_val_train_num],
            "data_type": [int],
            "block_size": [buffer_val_train_num] 
            }
        
        
        #Create Generators
        print("Creating training generator...")
        #give training data num
        """training_generator = DGTrain.DataGeneratorTrain(input_buffer_params,
                                                           output_buffer_params,
                                                           [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        training_generator = DGTrain.DataGeneratorTrainErrorTypeEdition(input_buffer_params,
                                                                        output_buffer_params,
                                                                        [list(range(buffer_val_train_num))] 
                                                )
        #Create Generators
        print("Creating validation generator...")
        #give valitdation data num
        """validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params,
                                                                       validation_output_buffer_params,
                                                                      [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        validation_generator = DGValidation.DataGeneratorValidationErrorTypeEdition(validation_input_buffer_params,
                                                  validation_output_buffer_params,
                                                  [list(range(buffer_val_train_num))] 
                                                )
        
        
        #''' <-----traing switch
        #Start training
        print("Strat training...")
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit_generator(generator = training_generator,
                                      epochs = 1000, #100 200 500 3000
                                      verbose = 2, #set visibility
                                      validation_data = validation_generator,
                                      callbacks = [model_checkpoint_callback],
                                     )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [ ]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [ ]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [ ]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [ ]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [ ]:
x=TestTranslate()
x.test_translate()